In [288]:
import yfinance as yf
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

# Import the optimise sublibrary
import scipy.optimize as sco
# Import cubic splines interpolation module
import scipy.interpolate as sci

In [289]:

msft = yf.Ticker("MSFT")
aapl_historical = msft.history(start="2020-06-02", end="2020-07-07")
#data = yf.download("AMZN AAPL GOOG ^GSPC",  start="2017-01-01", end="2018-01-01")

aapl_historical
data = yf.download("AMZN AAPL GOOG MSFT ",period="1y")
def df(data,pm):
    df=data[pm]
    return df
data=df(data,"Close")
data

[*********************100%%**********************]  4 of 4 completed


,AAPL,AMZN,GOOG,MSFT
Date,,,,
2022-09-08,154.460007,129.820007,109.419998,258.519989
2022-09-09,157.369995,133.270004,111.779999,264.459991
2022-09-12,163.429993,136.449997,111.870003,266.649994
2022-09-13,153.839996,126.820000,105.309998,251.990005
2022-09-14,155.309998,128.550003,105.870003,252.220001
...,...,...,...,...
2023-08-31,187.869995,138.009995,137.350006,327.760010
2023-09-01,189.460007,138.119995,136.800003,328.660004
2023-09-05,189.699997,137.270004,136.710007,333.549988


In [290]:
# Normalize stock data based on initial price
def normalize(data):
  x = data.copy()
  for i in x.columns[0:]:
    x[i] = x[i]/x[i][0]
  return x
normalize(data)

# Function to plot interactive plot
def interactive_plot(df, title):
  fig = px.line(title = title)
  for i in df.columns[0:]:
    fig.add_scatter(x = df.index, y = df[i], name = i)
  fig.show()


In [291]:
interactive_plot(data, 'Normalized Prices')

In [292]:
def daily_return(df):
    df_daily_return = df.copy()
    df_daily_return=df_daily_return.pct_change()
    for i in df.columns[0:].tolist():
        df_daily_return[i][0] = 0
    return df_daily_return
stocks_daily_return = daily_return(data)
stocks_daily_return


,AAPL,AMZN,GOOG,MSFT
Date,,,,
2022-09-08,0.000000,0.000000,0.000000,0.000000
2022-09-09,0.018840,0.026575,0.021568,0.022977
2022-09-12,0.038508,0.023861,0.000805,0.008281
2022-09-13,-0.058680,-0.070575,-0.058640,-0.054978
2022-09-14,0.009555,0.013641,0.005318,0.000913
...,...,...,...,...
2023-08-31,0.001172,0.021766,0.003067,-0.003133
2023-09-01,0.008463,0.000797,-0.004004,0.002746
2023-09-05,0.001267,-0.006154,-0.000658,0.014879


In [293]:
def generate_portfolios(df,n):
    rs=[]
    sts=[]
    for i in range(n):
        weights=np.random.random(len(df.columns[:(len(stocks_daily_return.columns))]))
        weights /= np.sum(weights)
        rs.append(np.sum(df.iloc[:,:(len(stocks_daily_return.columns))].mean()*weights)*252)
        sts.append(np.sqrt(np.dot(weights.T,np.dot(df.iloc[:,:(len(stocks_daily_return.columns))].cov()*252,weights))))
    rs=np.array(rs)
    sts=np.array(sts)
    return rs,sts,weights

In [294]:
a=generate_portfolios(stocks_daily_return,5000)
risk_free_rate=0
fig = go.Figure()
fig.add_trace(go.Scatter(x=a[1], 
                         y=a[0], 
                      #- Add color scale for sharpe ratio   
                      marker=dict(color=(a[1]-risk_free_rate)/(a[1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers'))
#- Add title/labels
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  coloraxis_colorbar=dict(title="Sharpe Ratio"))

In [295]:
data

,AAPL,AMZN,GOOG,MSFT
Date,,,,
2022-09-08,154.460007,129.820007,109.419998,258.519989
2022-09-09,157.369995,133.270004,111.779999,264.459991
2022-09-12,163.429993,136.449997,111.870003,266.649994
2022-09-13,153.839996,126.820000,105.309998,251.990005
2022-09-14,155.309998,128.550003,105.870003,252.220001
...,...,...,...,...
2023-08-31,187.869995,138.009995,137.350006,327.760010
2023-09-01,189.460007,138.119995,136.800003,328.660004
2023-09-05,189.699997,137.270004,136.710007,333.549988


In [296]:
stocks_daily_return

,AAPL,AMZN,GOOG,MSFT
Date,,,,
2022-09-08,0.000000,0.000000,0.000000,0.000000
2022-09-09,0.018840,0.026575,0.021568,0.022977
2022-09-12,0.038508,0.023861,0.000805,0.008281
2022-09-13,-0.058680,-0.070575,-0.058640,-0.054978
2022-09-14,0.009555,0.013641,0.005318,0.000913
...,...,...,...,...
2023-08-31,0.001172,0.021766,0.003067,-0.003133
2023-09-01,0.008463,0.000797,-0.004004,0.002746
2023-09-05,0.001267,-0.006154,-0.000658,0.014879


In [297]:
def ptf_stats(weights):
    weights = np.array(weights)
    ptf_r = np.sum(stocks_daily_return.mean() * weights) * 252
    ptf_std = np.sqrt(np.dot(weights.T, np.dot(stocks_daily_return.iloc[:,:(len(stocks_daily_return.columns))].cov() * 252, weights)))
    return {"return":ptf_r,"volatility": ptf_std, "sharpe":(ptf_r) / ptf_std}

# Minimise the negative value of the Sharpe ratio
def min_sharpe(weights):
    return -ptf_stats(weights)["sharpe"]
    # Write the constraint that the weights have to add up to 1

def minimize_volatility(weights):  
    # Note that we don't return the negative of volatility here because we 
    # want the absolute value of volatility to shrink, unlike sharpe.
    return ptf_stats(weights)["volatility"] 
def minimize_return(weights): 
    return -ptf_stats(weights)["return"]
    
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# Bound the weights (parameter inputs) to be within 0 and 1
bnds = tuple((0, 1) for x in range((len(data.columns))))


# Starting parameter (weights) list as equal distribution
starting_ws = (len(data.columns)) * [1. / (len(data.columns)),]

# Call the minimisation function
opts = sco.minimize(min_sharpe, starting_ws, method='SLSQP', bounds=bnds, constraints=cons)
# Inspect the results




In [298]:
cons

{'type': 'eq', 'fun': <function __main__.<lambda>(x)>}

In [299]:
optimal_sharpe_weights=opts['x'].round(4)
list(zip(data.columns[:len(stocks_daily_return)],list(optimal_sharpe_weights)))

[('AAPL', 0.0), ('AMZN', 0.0), ('GOOG', 0.2332), ('MSFT', 0.7668)]

In [300]:
# Obtain the optimal weights
weights_opt = opts['x'].round(3)
weights_opt

# Plug optimal weights into the statistics function



array([0.   , 0.   , 0.233, 0.767])

In [301]:

# Plug optimal weights into the statistics function
ptf_stats(weights_opt)

{'return': 0.29019602682137496,
 'volatility': 0.30340899538224286,
 'sharpe': 0.956451625489146}

In [302]:
optimal_stats = ptf_stats(optimal_sharpe_weights)
print(optimal_stats)

print('Optimal Portfolio Return: ', round(optimal_stats["return"]*100,4))
print('Optimal Portfolio Volatility: ', round(optimal_stats["volatility"]*100,4))
print('Optimal Portfolio Sharpe Ratio: ', round(optimal_stats["sharpe"],4))

{'return': 0.29019349802925287, 'volatility': 0.3034063491069861, 'sharpe': 0.9564516328790662}
Optimal Portfolio Return:  29.0193
Optimal Portfolio Volatility:  30.3406
Optimal Portfolio Sharpe Ratio:  0.9565


In [303]:



# Define a function that minimises portfolio variance
def min_var(weights):
    # Remember that variance is just standard deviation (volatility) squared
    return ptf_stats(weights)["volatility"]
# Call the optimisation fcuntion
opt_var = sco.minimize(min_var, starting_ws, method='SLSQP', bounds=bnds, constraints=cons)
# Inspect the results
opt_var



 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.27443093045230904
       x: [ 5.681e-01  5.350e-02  6.386e-02  3.146e-01]
     nit: 7
     jac: [ 2.743e-01  2.743e-01  2.742e-01  2.747e-01]
    nfev: 35
    njev: 7

In [304]:
# Obtain the optimal weights
weights_opt_var = opt_var['x'].round(3)
weights_opt_var


array([0.568, 0.054, 0.064, 0.315])

In [305]:

# Get the statistics for the absolute minimum variance portfolio
ptf_stats(weights_opt_var)

{'return': 0.2210518654673899,
 'volatility': 0.27470546911525906,
 'sharpe': 0.8046868021205739}

In [306]:


# Set up two conditions, one for the target return level and one for the sum of the portfolio weights
cons2 = ({'type': 'eq', 'fun': lambda x: ptf_stats(x)["return"] - r},
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# The boundary condition stays the same
bnds2 = tuple((0, 1) for x in weights_opt)
# Define a function that returns the volatility of a portfolio given a vector of weights
def min_port(weights):
    return ptf_stats(weights)["volatility"]

In [307]:
# Define a function to get the target returns and volatilities given a range of returns
def efficient_frontier(start_r, end_r, steps):
    target_rs = np.linspace(start_r, end_r, steps)
    target_stds = []
    for r in target_rs:
        cons2 = ({'type': 'eq', 'fun': lambda x: ptf_stats(x)["return"] - r},
                {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        bnds2 = tuple((0, 1) for x in a[2])
        opt_var = sco.minimize(min_var, starting_ws, method='SLSQP', bounds=bnds2, constraints=cons2)
        target_stds.append(opt_var['fun'])
    target_stds = np.array(target_stds)
    return target_rs, target_stds

target_rs, target_stds= efficient_frontier(a[0].min(),a[0].max(),50)

In [308]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=a[1], 
                         y=a[0], 
                      #- Add color scale for sharpe ratio   
                      marker=dict(color=(a[1]-risk_free_rate)/(a[1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers'))
fig.add_trace(go.Scatter(
    x=target_stds, y=target_rs, marker=dict(symbol="x",color=((target_rs - 0.01)/target_stds), 
                                  showscale=False, 
                                  size=7,colorscale="RdBu"
                                  ),mode="markers"
)

)
#- Add title/labels
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  coloraxis_colorbar=dict(title="Sharpe Ratio"))

In [309]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=a[1], 
                         y=a[0], 
                      #- Add color scale for sharpe ratio   
                      marker=dict(color=(a[1]-risk_free_rate)/(a[1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers'))
fig.add_trace(go.Scatter(
    x=target_stds, y=target_rs, mode="lines"
)

)
fig.add_trace(go.Scatter(x=[ptf_stats(optimal_sharpe_weights)['volatility']],y=[ptf_stats(optimal_sharpe_weights)['return']],marker=dict(symbol="x",size=10),mode="markers"))
#- Add title/labels
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  coloraxis_colorbar=dict(title="Sharpe Ratio"))

In [310]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=a[1], 
                         y=a[0], 
                      #- Add color scale for sharpe ratio   
                      marker=dict(color=(a[1]-risk_free_rate)/(a[1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers'))
fig.add_trace(go.Scatter(
    x=target_stds, y=target_rs, mode="lines"
)

)
fig.add_trace(go.Scatter(x=[ptf_stats(optimal_sharpe_weights)['volatility']],y=[ptf_stats(optimal_sharpe_weights)['return']],marker=dict(symbol="x",size=10),mode="markers"))
fig.add_trace(go.Scatter(x=[ptf_stats(weights_opt_var)['volatility']],y=[ptf_stats(weights_opt_var)['return']],marker=dict(symbol="x",size=10),mode="markers"))

#- Add title/labels
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  coloraxis_colorbar=dict(title="Sharpe Ratio"))

In [311]:

# Select the index of the absolute minimum variance portfolio
min_var = np.argmin(target_stds)

# Select expected volatilities for the stocks on the efficient frontier
ex_stds = target_stds[min_var:]

# Select expected returns for the stocks on the efficient frontier
ex_rs = target_rs[min_var:]

# Interpolate the B-spline representation of the data points
tck = sci.splrep(ex_stds, ex_rs)

In [312]:
# Define a continuously differentiable function f(x) for the efficient frontier
def f(x):
    return sci.splev(x, tck, der=0)

# Define te first derivate function of f(x)
def df(x):
    return sci.splev(x, tck, der=1)

# Set up a system of equations for the conditions that the CML has to satisfy
def cml_conditions(p, rf=0.01):
    eq1 = rf - p[0]
    eq2 = rf + p[1] * p[2] - f(p[2])
    eq3 = p[1] - df(p[2])
    return eq1, eq2, eq3

# Solve for the parameters of the Capital Market Line
cml = sco.fsolve(cml_conditions, [0, 0.5, 0.15])
# Create input variable for CML
cx = np.linspace(a[1].min(),a[1].max())

/home/batuhan-saylam/miniconda3/envs/tf/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:177: RuntimeWarning:

The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.



In [313]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=a[1], 
                         y=a[0], 
                      #- Add color scale for sharpe ratio   
                      marker=dict(color=(a[1]-risk_free_rate)/(a[1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers'))
fig.add_trace(go.Scatter(
    x=target_stds, y=target_rs, mode="lines"
)

)
fig.add_trace(go.Scatter(x=[ptf_stats(optimal_sharpe_weights)['volatility']],y=[ptf_stats(optimal_sharpe_weights)['return']],marker=dict(symbol="x",size=10),mode="markers"))
fig.add_trace(go.Scatter(x=cx,y=(cml[0] + cml[1] * cx),mode="lines"))

#- Add title/labels
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  coloraxis_colorbar=dict(title="Sharpe Ratio"))

In [314]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=target_stds, y=target_rs, mode="lines"
)

)
fig.add_trace(go.Scatter(x=[ptf_stats(optimal_sharpe_weights)['volatility']],y=[ptf_stats(optimal_sharpe_weights)['return']],marker=dict(symbol="x",size=10),mode="markers"))
fig.add_trace(go.Scatter(x=cx,y=(cml[0] + cml[1] * cx),mode="lines"))

#- Add title/labels
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  coloraxis_colorbar=dict(title="Sharpe Ratio"))